In [10]:
# !pip install "matplotlib>=3.6.0" "derivative~=0.6.0" "scikit-learn==1.1.3" "numpy>=1.20,<=1.26" "scipy>1.6.0,<=1.11.2" "pydmd>0.4,<=0.4.1" "optht~=0.2.0" "prettytable>3.0.0,<=3.9.0"
# pkg_resources 
!pip install pkg_resources

ERROR: Could not find a version that satisfies the requirement pkg_resources (from versions: none)
ERROR: No matching distribution found for pkg_resources


In [9]:
import sys
sys.path.append('../src')

%matplotlib inline
import pykoopman as pk
from pykoopman.common.examples import forced_duffing, rk4, sine_wave  # required for example system
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
np.random.seed(42)  # for reproducibility

import warnings
warnings.filterwarnings('ignore')



ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
import math

# State variables: x, y (position), theta (orientation), vx, vy (velocity), omega (angular velocity)
# Misaligned measurements: x2, y2 (Hall sensor outputs)
# Constants for Stribeck friction
delta = 0.1  # Empirically tuned small positive constant for Stribeck friction
alpha = 0.5  # Parameter for rational function approximation of Stribeck friction

# Dictionary of observables as lambda functions
observables = [
    # Basic state observables
    lambda x, y, theta, vx, vy, omega, x2, y2: x,  # Position x
    lambda x, y, theta, vx, vy, omega, x2, y2: y,  # Position y
    lambda x, y, theta, vx, vy, omega, x2, y2: theta,  # Orientation theta
    lambda x, y, theta, vx, vy, omega, x2, y2: vx,  # Velocity vx
    lambda x, y, theta, vx, vy, omega, x2, y2: vy,  # Velocity vy
    lambda x, y, theta, vx, vy, omega, x2, y2: omega,  # Angular velocity omega

    # Dry Friction (Coulomb): Sign functions, sgn(0) = 0
    lambda x, y, theta, vx, vy, omega, x2, y2: 1.0 if vx > 0 else (-1.0 if vx < 0 else 0.0),  # sgn(vx)
    lambda x, y, theta, vx, vy, omega, x2, y2: 1.0 if vy > 0 else (-1.0 if vy < 0 else 0.0),  # sgn(vy)
    lambda x, y, theta, vx, vy, omega, x2, y2: 1.0 if omega > 0 else (-1.0 if omega < 0 else 0.0),  # sgn(omega)

    # Stribeck Friction: Smooth approximations
    lambda x, y, theta, vx, vy, omega, x2, y2: vx * math.exp(-abs(vx) / delta),  # vx * e^(-|vx|/delta)
    lambda x, y, theta, vx, vy, omega, x2, y2: vy * math.exp(-abs(vy) / delta),  # vy * e^(-|vy|/delta)
    lambda x, y, theta, vx, vy, omega, x2, y2: omega * math.exp(-abs(omega) / delta),  # omega * e^(-|omega|/delta)
    lambda x, y, theta, vx, vy, omega, x2, y2: vx / (1 + alpha * abs(vx)),  # vx / (1 + alpha |vx|)
    lambda x, y, theta, vx, vy, omega, x2, y2: vy / (1 + alpha * abs(vy)),  # vy / (1 + alpha |vy|)
    lambda x, y, theta, vx, vy, omega, x2, y2: omega / (1 + alpha * abs(omega)),  # omega / (1 + alpha |omega|)
    lambda x, y, theta, vx, vy, omega, x2, y2: vx / (1 + alpha * vx ** 2),  # vx / (1 + alpha vx^2)
    lambda x, y, theta, vx, vy, omega, x2, y2: vy / (1 + alpha * vy ** 2),  # vy / (1 + alpha vy^2)
    lambda x, y, theta, vx, vy, omega, x2, y2: omega / (1 + alpha * omega ** 2),  # omega / (1 + alpha omega^2)

    # Gravity: Constant term
    lambda x, y, theta, vx, vy, omega, x2, y2: 1.0,  # Constant for gravity

    # External Magnetic Force: Position-dependent terms
    lambda x, y, theta, vx, vy, omega, x2, y2: x ** 2,  # x^2
    lambda x, y, theta, vx, vy, omega, x2, y2: y ** 2,  # y^2
    lambda x, y, theta, vx, vy, omega, x2, y2: x ** 3,  # x^3
    lambda x, y, theta, vx, vy, omega, x2, y2: y ** 3,  # y^3
    lambda x, y, theta, vx, vy, omega, x2, y2: 1 / (x ** 2 + y ** 2 + 1e-6),  # 1 / (x^2 + y^2), small constant to avoid division by zero
    lambda x, y, theta, vx, vy, omega, x2, y2: 1 / ((x ** 2 + y ** 2 + 1e-6) ** 1.5),  # 1 / (x^2 + y^2)^(3/2)

    # Hall Sensor Misalignment: Direct inclusion of misaligned measurements
    lambda x, y, theta, vx, vy, omega, x2, y2: x2,  # Misaligned x2
    lambda x, y, theta, vx, vy, omega, x2, y2: y2,  # Misaligned y2
    lambda x, y, theta, vx, vy, omega, x2, y2: x2 ** 2,  # x2^2
    lambda x, y, theta, vx, vy, omega, x2, y2: y2 ** 2,  # y2^2
    lambda x, y, theta, vx, vy, omega, x2, y2: x2 * y2,  # x2 * y2
]

# Corresponding observable names for interpretability
observable_names = [
    # Basic state observables
    lambda s: "x",
    lambda s: "y",
    lambda s: "theta",
    lambda s: "vx",
    lambda s: "vy",
    lambda s: "omega",

    # Dry Friction
    lambda s: "sgn(vx)",
    lambda s: "sgn(vy)",
    lambda s: "sgn(omega)",

    # Stribeck Friction
    lambda s: f"vx * e^(-|vx|/{delta})",
    lambda s: f"vy * e^(-|vy|/{delta})",
    lambda s: f"omega * e^(-|omega|/{delta})",
    lambda s: f"vx / (1 + {alpha} |vx|)",
    lambda s: f"vy / (1 + {alpha} |vy|)",
    lambda s: f"omega / (1 + {alpha} |omega|)",
    lambda s: f"vx / (1 + {alpha} vx^2)",
    lambda s: f"vy / (1 + {alpha} vy^2)",
    lambda s: f"omega / (1 + {alpha} omega^2)",

    # Gravity
    lambda s: "1",

    # External Magnetic Force
    lambda s: "x^2",
    lambda s: "y^2",
    lambda s: "x^3",
    lambda s: "y^3",
    lambda s: "1 / (x^2 + y^2)",
    lambda s: "1 / (x^2 + y^2)^(3/2)",

    # Hall Sensor Misalignment
    lambda s: "x2",
    lambda s: "y2",
    lambda s: "x2^2",
    lambda s: "y2^2",
    lambda s: "x2 * y2",
]